In [ ]:
# Python version check
import sys
!{sys.executable} --version

In [ ]:
# version check
import sklearn
print('이 노트북에 사용된 scikit-learn 버전: {}.'.format(sklearn.__version__))

In [ ]:
import sqlite3 as sq

In [ ]:
from sql import *
from sql.aggregate import *
from sql.conditionals import *

In [16]:
import numpy as np
import random
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
%matplotlib inline

Step 0. Glossary 
    - trx07 
    - trx08
    - trx09
    - upjong
    - cus

Step 1. Data Extraction 

In [174]:
# make sample

num_txn = 1000000 # 1/100 of real data 
num_cus = 40000 # 1/100 of real data
num_mct = 100 

array01 = np.random.randint(0, num_cus, size=(num_txn, 1) ) # CLNN

In [175]:
array02 = np.random.randint(180701, 180731, size=(num_txn, 1)) # TS_D # 승인날짜
times = pd.date_range('2018-07-01', periods=31, freq='1440min')
array02 = np.random.choice(times, size=num_txn, replace=True, p=None)
array02

array(['2018-07-27T00:00:00.000000000', '2018-07-25T00:00:00.000000000',
       '2018-07-04T00:00:00.000000000', ...,
       '2018-07-05T00:00:00.000000000', '2018-07-21T00:00:00.000000000',
       '2018-07-08T00:00:00.000000000'], dtype='datetime64[ns]')

In [176]:
array03 = np.random.randint(0, 100, size=(num_txn, 1)) # FINAL_CD # 업종코드 

In [205]:
array04a = np.random.randn(num_txn, 1) # SAA # 승인금액
array04 = array04^2 ## 무엇?
array04

array([[873],
       [780],
       [734],
       ...,
       [ 98],
       [436],
       [735]], dtype=int32)

In [178]:
array05old = np.random.randint(0, 424, size=(num_txn, 1)) # MCT_ADM_GDS_APB_CD # 서울 행정동 코드
array05 = np.random.randint(0, 424, size=(num_txn, 1))

In [179]:
# array06a = np.random.randint(1, 2, size=(round(num_txn*5/7), 1)) # DOF_TF # 평일 or 휴일
# array06b = np.random.randint(0, 1, size=(round(num_txn*2/7), 1)) # DOF_TF # 평일 or 휴일
# array06 = np.append(array06a, array06b, axis=0)
# random.shuffle(array06)

In [181]:
# array06ab.shape[0] # fraction 했기 때문에 최종 number of rows 가 num_txn과 동일한지 확인
array06.shape

(1000000,)

In [180]:
# 다른 방법
array06 = np.random.choice(2, size=num_txn, p=[5/7, 2/7]) # 2 미만의 정수 중 p의 확률로 랜덤 선택
array06

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [183]:
array07 = np.random.randint(0, 6, size=(num_txn, 1)) # DAW_CCD # 요일코드 # array06과 맞추기 어려워서 임시로 랜덤

In [184]:
array08 = np.random.randint(0, num_mct, size=(num_txn, 1)) # MCT_N # 가맹점 번호

In [ ]:
array09 = np.random.randint(1, 240000, size=(num_txn, 1)) # TIME # 24h 시간      

In [206]:
array09a = np.random.randn(num_txn, 1)
array09 = array09^2
array09

array([[128095],
       [115603],
       [ 21335],
       ...,
       [ 95755],
       [ 12689],
       [ 64144]], dtype=int32)

In [207]:
df = pd.DataFrame( np.column_stack([array01, array02, array03, array04, array05, array06, array07, array08, array09])
                  , columns = ['CLNN', 'TS_D', 'FINAL_CD', 'SAA', 'MCT_ADM_GDS_APB_CD', 'DOF_TF', 'DAW_CCD', 'MCT_N', 'TIME'])

df.head()

TypeError: invalid type promotion

### GroupBy 객체 만들기

In [153]:
groupedByCLNN = df.groupby('CLNN')

In [154]:
groupedByCLNN.describe()['SAA'].head() # 계산시 오래걸림

KeyboardInterrupt: 

In [ ]:
groupedByCLNN.sum()['SAA'].head()

In [ ]:
# 2개 기준으로 Group By 하기
gb = df.groupby(['CLNN', 'MCT_ADM_GDS_APB_CD']) # 고객별 & 행정동별
gb2 = gb.sum()['SAA'] # 금액 합
gb2

In [ ]:
# 2개 기준으로 Group By 하기
gb3 = df.groupby(['CLNN', 'MCT_ADM_GDS_APB_CD']).sum()['SAA'] # 고객별 & 행정동별 합 chaining
gb3

In [71]:
df.sort_values(['MCT_ADM_GDS_APB_CD', 'SAA'], ascending=False).groupby('CLNN').head()

,CLNN,TS_D,FINAL_CD,SAA,MCT_ADM_GDS_APB_CD,DOF_TF,DAW_CCD,MCT_N,TIME
36061,28830,180723,14,999,423,1,5,89661,167687
352984,3143,180721,29,999,423,1,1,55450,163091
489033,12160,180717,88,999,423,1,5,62823,723
503383,7262,180701,72,999,423,1,3,74335,23740
864531,9185,180717,14,999,423,0,0,20508,201150
367647,35615,180722,93,998,423,1,5,4099,19169
602991,33468,180725,79,998,423,1,2,66336,211472
863896,8331,180724,65,998,423,0,0,7985,113582
42367,18305,180715,62,996,423,1,2,9410,75848
706865,14157,180704,71,996,423,1,1,13492,184048


### GroupBy 객체를 sorting 하기

In [107]:
# 고객당 평균 몇 개의 동에서 거래가 이루어졌는가?
grouped = df.groupby(['CLNN'], sort=False).count()['MCT_ADM_GDS_APB_CD']
print(grouped.head(15))
print(np.mean(grouped[:2]) )

CLNN
14737    22
33245    25
19315    21
716      24
31275    23
19654    27
23452    27
22061    28
19893    30
12197    24
2036     20
34320    23
3455     23
20922    21
2153     21
Name: MCT_ADM_GDS_APB_CD, dtype: int64
23.5


In [94]:
grouped = df.groupby(['CLNN', 'MCT_ADM_GDS_APB_CD'], sort=False).sum() # 여기에서 sort=False 가 중요한 것 같음
grouped.sort_values('SAA', ascending=False)['SAA'] # multiple 기준으로 sorting은 안되는듯

CLNN   MCT_ADM_GDS_APB_CD
10371  186                   3013
37069  197                   2922
15266  164                   2818
4572   363                   2810
16064  11                    2747
9038   107                   2703
10311  109                   2623
35094  315                   2614
29163  54                    2610
14073  386                   2567
3719   395                   2560
2572   90                    2554
37011  134                   2514
14756  117                   2502
6396   157                   2502
34823  189                   2498
32355  370                   2490
39665  93                    2489
34671  409                   2473
30806  417                   2472
36379  51                    2470
977    8                     2419
20678  26                    2374
23451  401                   2356
26901  212                   2337
16778  107                   2329
15409  10                    2327
30576  248                   2298
21719  351            

In [96]:
grouped.sort_values('CLNN', ascending=True) # NOT WORKING !! 처음 group by 한 원래 기준으로 sorting도 안되는듯

KeyError: 'CLNN'

In [58]:
gb2.apply(lambda _df: _df.sort_values(by=['SAA']))

AttributeError: 'int' object has no attribute 'sort_values'

In [ ]:
df.pivot('MCT_ADM_GDS_APB_CD', 'FINAL_CD', 'CLNN')

In [ ]:
df.sort_values(by=['MCT_ADM_GDS_APB_CD', 'TIME'], ascending=True).head()

### 조건에 맞는 일부 원소만 추출하기

### 문자열을 숫자로 type 바꾸기

### 시간정보를 파싱하기

In [ ]:
# upjong

num_upjong = 2786049
upjong = pd.DataFrame(np.random.randint(0, 100, size=(num_upjong, 3)))
upjong.columns = ['MCT_N', 'FINAL_CD', 'FINAL_CD_NM']
upjong.head()

In [ ]:
grouped = upjong.groupby('MCT_N')

In [ ]:
grouped.head()

In [ ]:
grouped.count().head()

In [ ]:
grouped.describe()['SAA'].head()

In [ ]:
# file read in memory 
# takes a long time so beware session failure
trx07 = pd.read_csv('') 

In [ ]:
# file read in memory 
# takes a long time so beware session failure
trx08 = pd.read_csv('')

In [ ]:
# file read in memory 
# takes a long time so beware session failure
trx09 = pd.read_csv('') 

In [ ]:
# check if data is succesfully loaded in memory
# count rows


In [ ]:
# check data


Step 2. Data Wrangling

In [ ]:
# count trx, group by dong

# Make GroupBy object
groupByDong = trx07.groupby('dong')

In [ ]:
groupedByDong.count()
groupedByDong.sum()

만약 특정 변수에 대해서만 그룹별 요약/집계를 하고 싶다면 해당 변수를 indexing해주면 되며, 한개 변수에 대해서만 GroupBy 집계를 하면 반환되는 결과는 Series가 됩니다. 한개 변수에 대해 GroupBy 집계해서 나온 Series를 DataFrame으로 만들고 싶으면 pd.DataFrame() 를 사용해서 집계 결과를 DataFrame으로 변환해주면 됩니다. 

In [ ]:
groupedByDong.sum()['amount'] # Series

pd.DataFrame(groupedByDong.sum()['amount']) # DataFrame 

In [ ]:
groupedByDong.describe()['amount']

필요로 하는 집계함수가 pandas GroupBy methods에 없는 경우 사용자 정의 함수를 정의해서 집계에 사용할 수 있습니다. IQR(Inter-Quartile Range, Q3 - Q1) 를 사용자 정의 함수로 정의하고, 이를 grouped.aggregate() 혹은 grouped.agg() 의 괄호 안에 넣어서 그룹 별로 IQR를 계산해보겠습니다. 

In [ ]:
def iqr_func(x):

    q3, q1 = np.percentile(x, [75, 25])

    iqr = q3 - q1

    return iqr

In [ ]:
groupedByDong.agg(iqr_func) # groupedByDong.aggregate(iqr_func) 도 같은 결과